In [67]:
import pandas as pd
import numpy as np
import pyodbc as po
import os
import glob
import warnings
warnings.filterwarnings("ignore")
from fast_to_sql import fast_to_sql as fts

ruta_origen = r"C:\Users\wymd_\Documents\Documentos\DataPath\Python\data\proyecto"

In [37]:
columns_names={'categories':['CATEGORY_ID', 'CATEGORY_DEPARTAMENT_ID','CATEGORY_NAME'],
               'orders':['ORDER_ID', 'ORDER_DATE','ORDER_CUSTOMER_ID','ORDER_STATUS'],
               'customer':['CUSTOMER_ID', 'CUSTOMER_FNAME','CUSTOMER_LNAME','CUSTOMER_EMAIL','CUSTOMER_PASSWORD','CUSTOMER_STREET','CUSTOMER_STATE','CUSTOMER_CITY','CUSTOMER_ZIPCODE'],
               'order_items':['ORDER_ITEM_ID', 'ORDER_ITEM_ORDER_ID','ORDER_ITEM_PRODUCT_ID','ORDER_ITEM_QUANTITY','ORDER_ITEM_SUBTOTAL','ORDER_ITEM_PRODUCT_PRICE'],
               'products':['PRODUCT_ID','PRODUCT_CATEGORY_ID','PRODUCT_NAME','PRODUCT_DESCRIPTION','PRODUCT_PRICE','PRODUCT_IMAGE'],
               'departments':['DEPARTMENT_ID','DEPARTMENT_NAME']
}

In [38]:
def read_dataframe(ruta,table):
    
    df_name=pd.read_csv(ruta+ "/" +table,sep='|',header=None,names=columns_names[table])

    return df_name

In [39]:
tables = list(columns_names.keys())
dict_tables=dict()

for i in tables:
   dict_tables[i]=read_dataframe(ruta_origen,i)

In [40]:
print(dict_tables['departments'])

   DEPARTMENT_ID DEPARTMENT_NAME
0              2         Fitness
1              3        Footwear
2              4         Apparel
3              5            Golf
4              6        Outdoors
5              7        Fan Shop


# Analisis

In [41]:
dict_tables['orders'].head()


,ORDER_ID,ORDER_DATE,ORDER_CUSTOMER_ID,ORDER_STATUS
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


In [42]:
dict_tables['order_items'].head()

,ORDER_ITEM_ID,ORDER_ITEM_ORDER_ID,ORDER_ITEM_PRODUCT_ID,ORDER_ITEM_QUANTITY,ORDER_ITEM_SUBTOTAL,ORDER_ITEM_PRODUCT_PRICE
0,1,1,957,1,299.98,299.98
1,2,2,1073,1,199.99,199.99
2,3,2,502,5,250.00,50.00
3,4,2,403,1,129.99,129.99
4,5,4,897,2,49.98,24.99


In [43]:
# Revisamos los valores nulos o missings!
dict_tables['orders'].isnull().sum()/dict_tables['orders'].shape[0]

ORDER_ID             0.0
ORDER_DATE           0.0
ORDER_CUSTOMER_ID    0.0
ORDER_STATUS         0.0
dtype: float64

In [44]:
dict_tables['orders'].describe()

,ORDER_ID,ORDER_CUSTOMER_ID
count,68883.000000,68883.000000
mean,34442.000000,6216.571099
std,19884.953633,3586.205241
min,1.000000,1.000000
25%,17221.500000,3122.000000
50%,34442.000000,6199.000000
75%,51662.500000,9326.000000
max,68883.000000,12435.000000


In [49]:
df_base = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(dict_tables['orders'], 
          dict_tables['order_items'],left_on='ORDER_ID', right_on='ORDER_ITEM_ORDER_ID',how='left'),
          dict_tables['customer'],left_on='ORDER_CUSTOMER_ID', right_on='CUSTOMER_ID',how='left'),
          dict_tables['products'],left_on='ORDER_ITEM_PRODUCT_ID', right_on='PRODUCT_ID',how='left'),
          dict_tables['categories'],left_on='PRODUCT_CATEGORY_ID', right_on='CATEGORY_ID',how='left'),
          dict_tables['departments'],left_on='CATEGORY_DEPARTAMENT_ID', right_on='DEPARTMENT_ID',how='left')
df_base.head()

,ORDER_ID,ORDER_DATE,ORDER_CUSTOMER_ID,ORDER_STATUS,ORDER_ITEM_ID,ORDER_ITEM_ORDER_ID,ORDER_ITEM_PRODUCT_ID,ORDER_ITEM_QUANTITY,ORDER_ITEM_SUBTOTAL,ORDER_ITEM_PRODUCT_PRICE,...,PRODUCT_CATEGORY_ID,PRODUCT_NAME,PRODUCT_DESCRIPTION,PRODUCT_PRICE,PRODUCT_IMAGE,CATEGORY_ID,CATEGORY_DEPARTAMENT_ID,CATEGORY_NAME,DEPARTMENT_ID,DEPARTMENT_NAME
0,1,2013-07-25 00:00:00.0,11599,CLOSED,1.0,1.0,957.0,1.0,299.98,299.98,...,43.0,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,43.0,7.0,Camping & Hiking,7.0,Fan Shop
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT,2.0,2.0,1073.0,1.0,199.99,199.99,...,48.0,Pelican Sunstream 100 Kayak,NaN,199.99,http://images.acmesports.sports/Pelican+Sunstr...,48.0,7.0,Water Sports,7.0,Fan Shop
2,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT,3.0,2.0,502.0,5.0,250.00,50.00,...,24.0,Nike Men's Dri-FIT Victory Golf Polo,NaN,50.00,http://images.acmesports.sports/Nike+Men%27s+D...,24.0,5.0,Women's Apparel,5.0,Golf
3,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT,4.0,2.0,403.0,1.0,129.99,129.99,...,18.0,Nike Men's CJ Elite 2 TD Football Cleat,NaN,129.99,http://images.acmesports.sports/Nike+Men%27s+C...,18.0,4.0,Men's Footwear,4.0,Apparel
4,3,2013-07-25 00:00:00.0,12111,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df_base_new = df_base[['ORDER_ID', 'ORDER_DATE','ORDER_STATUS','ORDER_ITEM_ID', 'ORDER_ITEM_ORDER_ID',
              'ORDER_ITEM_PRODUCT_ID','ORDER_ITEM_QUANTITY','ORDER_ITEM_SUBTOTAL','ORDER_ITEM_PRODUCT_PRICE',
              'CUSTOMER_ID', 'CUSTOMER_FNAME','CUSTOMER_LNAME','CUSTOMER_EMAIL',
              'PRODUCT_ID','PRODUCT_CATEGORY_ID','PRODUCT_NAME','PRODUCT_PRICE','CATEGORY_NAME','DEPARTMENT_NAME']]

In [55]:
df_base_new.dtypes

ORDER_ID                      int64
ORDER_DATE                   object
ORDER_STATUS                 object
ORDER_ITEM_ID               float64
ORDER_ITEM_ORDER_ID         float64
ORDER_ITEM_PRODUCT_ID       float64
ORDER_ITEM_QUANTITY         float64
ORDER_ITEM_SUBTOTAL         float64
ORDER_ITEM_PRODUCT_PRICE    float64
CUSTOMER_ID                   int64
CUSTOMER_FNAME               object
CUSTOMER_LNAME               object
CUSTOMER_EMAIL               object
PRODUCT_ID                  float64
PRODUCT_CATEGORY_ID         float64
PRODUCT_NAME                 object
PRODUCT_PRICE               float64
CATEGORY_NAME                object
DEPARTMENT_NAME              object
dtype: object

In [60]:
df_base_new['ORDER_DATE'] = pd.to_datetime(df_base_new['ORDER_DATE'])
df_base_new['ORDER_PERIOD'] = df_base_new['ORDER_DATE'].dt.strftime("%Y%m")
df_base_new.head()

,ORDER_ID,ORDER_DATE,ORDER_STATUS,ORDER_ITEM_ID,ORDER_ITEM_ORDER_ID,ORDER_ITEM_PRODUCT_ID,ORDER_ITEM_QUANTITY,ORDER_ITEM_SUBTOTAL,ORDER_ITEM_PRODUCT_PRICE,CUSTOMER_ID,CUSTOMER_FNAME,CUSTOMER_LNAME,CUSTOMER_EMAIL,PRODUCT_ID,PRODUCT_CATEGORY_ID,PRODUCT_NAME,PRODUCT_PRICE,CATEGORY_NAME,DEPARTMENT_NAME,ORDER_PERIOD
0,1,2013-07-25,CLOSED,1.0,1.0,957.0,1.0,299.98,299.98,11599,Mary,Malone,XXXXXXXXX,957.0,43.0,Diamondback Women's Serene Classic Comfort Bi,299.98,Camping & Hiking,Fan Shop,201307
1,2,2013-07-25,PENDING_PAYMENT,2.0,2.0,1073.0,1.0,199.99,199.99,256,David,Rodriguez,XXXXXXXXX,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,Water Sports,Fan Shop,201307
2,2,2013-07-25,PENDING_PAYMENT,3.0,2.0,502.0,5.0,250.00,50.00,256,David,Rodriguez,XXXXXXXXX,502.0,24.0,Nike Men's Dri-FIT Victory Golf Polo,50.00,Women's Apparel,Golf,201307
3,2,2013-07-25,PENDING_PAYMENT,4.0,2.0,403.0,1.0,129.99,129.99,256,David,Rodriguez,XXXXXXXXX,403.0,18.0,Nike Men's CJ Elite 2 TD Football Cleat,129.99,Men's Footwear,Apparel,201307
4,3,2013-07-25,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,12111,Amber,Franco,XXXXXXXXX,NaN,NaN,NaN,NaN,NaN,NaN,201307


In [64]:
result = df_base_new.groupby('DEPARTMENT_NAME').aggregate({'ORDER_ID':'count','PRODUCT_PRICE':['min','max']})
print(result)

                ORDER_ID PRODUCT_PRICE         
                   count           min      max
DEPARTMENT_NAME                                
Apparel            46797         59.99   299.99
Fan Shop           66332         29.99   399.98
Fitness             2122         22.00   999.99
Footwear           14525         27.99  1999.99
Golf               33220         25.00   199.99
Outdoors            9202          9.99   599.99


In [63]:
result = df_base_new['ORDER_PERIOD'].aggregate(['min','max'])
print(result)

min    201307
max    201407
Name: ORDER_PERIOD, dtype: object


In [69]:
#Variables de conexion
server = 'DESKTOP-5S41MDR'
database = 'proyecto'

#Conexion string
conexion = po.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' +
           server+';DATABASE='+database+';Trusted_Connection=yes;MARS_Connection=yes')

conexion.autocommit=True
cursor =conexion.cursor()

sql = "TRUNCATE TABLE dbo.ordenes"
print(sql)
cursor.execute(sql)

fts.fast_to_sql(df_base_new['ORDER_PERIOD']>= '201309', "ordenes", conexion, if_exists="replace", temp=False)

'create table [dbo].[ordenes]\n(\n\t[ORDER_PERIOD] bit\n)'